# Image Communication notebook

It can be used to get either the predictions or the timing depending on the program you run on Arduino.

In [1]:
import serial
import time
import ast
import tensorflow as tf
import numpy as np

### Communication for inference results

The next functions are built to send the images to the Arduino board and receive back the predictions for tinyML and LR ("main_program"). Those results are saved as a .txt file 

In [2]:
def send_image(serial_port, image_path):
    with open(image_path, 'rb') as file:
        image_data = file.read()
        serial_port.write(image_data)
        time.sleep(0.1)  


In [5]:
def sending(beginning_image, ending_image):
    
    # Select the port to which the Arduino is connected
    arduino_port = serial.Serial('COM7', 115200, timeout=5)
    time.sleep(0.5)
    
    for i in range(beginning_image,ending_image):
        # Select the path of the binary images
        image_path = r'C:\Users\Iñaki\Documents\IMDEA\test_images_new\test_image_'+str(i)+'.bin'
        send_image(arduino_port, image_path)
    arduino_port.close()
 
    

In [10]:
def receiving(i):
    data = ''

    # The number below is appropiate to receive the output for packets of 33 images. If the number of images is different, it should be changed
    expected_length = 190
    # Select the port to which the Arduino is connected
    arduino_port = serial.Serial('COM7', 115200, timeout=30)
    time.sleep(0.5)
    while len(data) < expected_length:
        received = arduino_port.read(expected_length - len(data)).decode('utf-8')
        data += received
    arduino_port.close()

    
    tuple_list = [tuple(map(int, x.strip('[]').split(','))) for x in data.split('][')]
    sorted_tuple_list = sorted(tuple_list, key=lambda x: x[0])
    # Remenber to change the number below to the number of images received
    sorted_tuple_list = [(x[0] + i *25, *x[1:]) for x in sorted_tuple_list]
    
    return sorted_tuple_list

In [11]:
def totti(first_image, last_image, i):
    sending(first_image,last_image)
    lista = receiving(i)
    return lista

In [13]:
received_data = []
start_time = time.time()
for i in range(400):
    received_data.append(totti(25 * i, 25 * (i+1), i))
    
received_data_list = [item for sublist in received_data for item in sublist]
end_time = time.time()
print(received_data_list)
print(end_time - start_time)

# Save the received data in a txt file. Choose the path where you want to save it
file_path = r'C:\Users\Iñaki\Documents\IMDEA\Arduino_results.txt'
with open(file_path, 'w') as file:
    for item in received_data_list:
        file.write(f'{item}\n')

[(0, 1, 3), (1, 1, 8), (2, 1, 8), (3, 0, 8), (4, 1, 6), (5, 1, 6), (6, 1, 1), (7, 0, 6), (8, 1, 3), (9, 1, 1), (10, 1, 0), (11, 1, 9), (12, 1, 5), (13, 1, 7), (14, 1, 9), (15, 1, 8), (16, 0, 7), (17, 1, 7), (18, 1, 8), (19, 1, 6), (20, 1, 7), (21, 0, 0), (22, 1, 4), (23, 1, 9), (24, 1, 5), (25, 1, 2), (26, 0, 4), (27, 1, 0), (28, 1, 9), (29, 1, 6), (30, 1, 6), (31, 1, 5), (32, 0, 4), (33, 0, 5), (34, 1, 9), (35, 0, 1), (36, 1, 4), (37, 1, 1), (38, 1, 9), (39, 1, 5), (40, 1, 4), (41, 1, 6), (42, 1, 5), (43, 1, 6), (44, 1, 0), (45, 1, 9), (46, 0, 3), (47, 1, 9), (48, 1, 7), (49, 1, 6), (50, 1, 9), (51, 1, 8), (52, 0, 7), (53, 0, 3), (54, 1, 8), (55, 1, 8), (56, 0, 7), (57, 1, 3), (58, 0, 2), (59, 0, 3), (60, 1, 7), (61, 0, 5), (62, 1, 6), (63, 0, 1), (64, 1, 6), (65, 1, 2), (66, 1, 1), (67, 1, 2), (68, 0, 3), (69, 1, 7), (70, 0, 2), (71, 1, 6), (72, 1, 8), (73, 1, 8), (74, 0, 9), (75, 1, 2), (76, 1, 9), (77, 1, 3), (78, 1, 5), (79, 1, 8), (80, 1, 8), (81, 1, 1), (82, 1, 1), (83, 1, 7), (

### Communication for time measurements.

The next cells must be used with the "time_measurements" Arduino program. Similar functions as the ones used above. 

In [ ]:
def sending(beginning_image, ending_image):
    
    arduino_port = serial.Serial('COM3', 115200, timeout=5)
    time.sleep(0.5)
    
    for i in range(beginning_image,ending_image):
        image_path = r'C:\Users\Iñaki\Documents\IMDEA\test_images_new\test_image_'+str(i)+'.bin'
        send_image(arduino_port, image_path)
        #print("image "+str(i)+" sent")
    arduino_port.close()

In [ ]:
def receiving_time(i):
    data = ''
    expected_length = 500
    arduino_port = serial.Serial('COM3', 115200, timeout=30)
    time.sleep(0.5)
    while len(data) < expected_length:
        received = arduino_port.read(expected_length - len(data)).decode('utf-8')
        data += received
    arduino_port.close()
    
    tuple_list = [tuple(map(int, filter(None, x.strip('[]').split(',')))) for x in data.split('][')]

    sorted_tuple_list = sorted(tuple_list, key=lambda x: x[0])
    sorted_tuple_list = [(x[0] + i *25, *x[1:]) for x in sorted_tuple_list]
    
    
    return sorted_tuple_list

In [ ]:
def totti_time(first_image, last_image, i):
    sending(first_image,last_image)
    lista = receiving_time(i)
    return lista

In [ ]:
received_data = []
start_time = time.time()
for i in range(4):
    received_data.append(totti_time(25 * i, 25 * (i+1), i))
    
received_data_list = [item for sublist in received_data for item in sublist]
end_time = time.time()
print(received_data_list)
print(end_time - start_time)

file_path = r'C:\Users\Iñaki\Documents\IMDEA\Arduino_time_results.txt'
with open(file_path, 'w') as file:
    for item in received_data_list:
        file.write(f'{item}\n')

In [ ]:
element1_list, element2_list, element3_list = zip(*received_data_list)

average_element1 = sum(element1_list) / len(element1_list)
average_element2 = sum(element2_list) / len(element2_list)
average_element3 = sum(element3_list) / len(element3_list)

print("Average of Element 1:", average_element1)
print("Average of Element 2:", average_element2)
print("Average of Element 3:", average_element3)